In [9]:
import os
import uuid
import pandas as pd

from dotenv import load_dotenv

from neomodel import (
    StructuredNode,
    StringProperty,
    IntegerProperty,
    FloatProperty,
    DateTimeProperty,
    RelationshipTo,
)

load_dotenv()

True

In [10]:
URI=os.getenv("NEO4J_URI")
USERNAME=os.getenv("NEO4J_USERNAME")
PASSWORD=os.getenv("NEO4J_PASSWORD")

## Read Dataset

In [11]:
dietary_habit = pd.read_csv('../data/cleaned dietary habit.csv')
dietary_habit.head()

,Date,Size,Food_Name,Calories,Carbohydrates,Fat,Protein,Sodium,Sugar
0,03/04/24,200 gram,White Rice,300.0,80.0,1.0,6.0,0.0,0.0
1,03/04/24,24 gram,Crispy Fried Chicken with Chili,132.0,17.0,6.0,2.0,324.0,4.0
2,04/04/24,2 piece,Fried Noodles,820.0,116.0,32.0,16.0,0.0,14.0
3,04/04/24,1 container,Padang Rice,664.0,70.0,15.0,5.0,0.0,0.0
4,04/04/24,0.8 container,Padang Rice,531.0,56.0,12.0,4.0,0.0,0.0


In [12]:
physical_activity = pd.read_csv('../data/Physical Activity.csv')
physical_activity.head()

,Timestamp,Daily Steps,Distance (km),Calories Burned
0,13/04/24,4776,3.72,238
1,14/04/24,5337,3.94,232
2,15/04/24,9597,8.18,544
3,16/04/24,6842,5.23,296
4,17/04/24,7156,5.48,309


In [13]:
physiological_parameter = pd.read_csv('../data/Physiological Parameter.csv')
physiological_parameter.head()

,Timestamp,Heart Rate,Blood Pressure,Blood Oxygen
0,19/04/24 15:20,69,78/109,95
1,19/04/24 20:09,69,78/108,94
2,20/04/24 15:40,99,84/111,97
3,21/04/24 08:15,82,81/112,96
4,22/04/24 14:30,75,80/110,97


In [14]:
sleep_duration = pd.read_csv('../data/Sleep Duration.csv')
sleep_duration.head()

,Timestamp,Hours,Minutes
0,05/10/24,7,15
1,06/10/24,8,30
2,07/10/24,6,45
3,08/10/24,7,20
4,09/10/24,7,50


## Convert Timestamp Based Neo4J Object

In [15]:
dietary_habit['Date'] = pd.to_datetime(dietary_habit['Date'], format='%d/%m/%y')
physical_activity['Timestamp'] = pd.to_datetime(physical_activity['Timestamp'], format='%d/%m/%y')
physiological_parameter['Timestamp'] = pd.to_datetime(physiological_parameter['Timestamp'], format='%d/%m/%y %H:%M')
sleep_duration['Timestamp'] = pd.to_datetime(sleep_duration['Timestamp'], format='%d/%m/%y')

## Define Nodes and Relationships

In [16]:
class User(StructuredNode):
    _id = StringProperty(unique_index=True)
    name = StringProperty(required=True)
    age = IntegerProperty(required=True)
    height = FloatProperty(required=True)
    weight = FloatProperty(required=True)

    has_done = RelationshipTo('PhysicalActivity', 'HAS_DONE')
    has_consumed = RelationshipTo('Food', 'HAS_CONSUMED')
    has_monitored = RelationshipTo('PhysiologicalParameter', 'HAS_MONITORED')
    has_sleep = RelationshipTo('SleepDuration', 'HAS_SLEEP')

class Food(StructuredNode):
    _id = StringProperty(unique_index=True)
    name = StringProperty(required=True)
    serving_size = StringProperty(required=True)
    calories = FloatProperty(required=True)
    carbohydrates = FloatProperty(required=True)
    protein = FloatProperty(required=True)
    fat = FloatProperty(required=True)
    sodium = FloatProperty(required=True)
    sugar = FloatProperty(required=True)
    timestamp = DateTimeProperty(required=True)

class PhysicalActivity(StructuredNode):
    _id = StringProperty(unique_index=True)
    daily_steps = IntegerProperty(required=True)
    distance = FloatProperty(required=True)
    calories_burned = FloatProperty(required=True)
    timestamp = DateTimeProperty(required=True)

class PhysiologicalParameter(StructuredNode):
    _id = StringProperty(unique_index=True)
    heart_rate = IntegerProperty(required=True)
    blood_pressure = StringProperty(required=True)  
    blood_oxygen = FloatProperty(required=True)
    timestamp = DateTimeProperty(required=True)

class SleepDuration(StructuredNode):
    _id = StringProperty(unique_index=True)
    hours = IntegerProperty(required=True)
    minutes = IntegerProperty(required=True)
    timestamp = DateTimeProperty(required=True)

## Populate Graph with Dataset

In [17]:
user = User(
    _id=str(uuid.uuid4()),
    name="Adrian Putra Pratama Badjideh",
    age=20,
    height=172.0,  
    weight=85.0   
)
user.save()

print("Success populating user data...")

Success populating user data...


In [18]:
for index, row in dietary_habit.iterrows():
    unique_id = str(uuid.uuid4())

    food = Food(
        _id=unique_id,
        name=row['Food_Name'],
        serving_size=row['Size'],
        calories=row['Calories'],
        carbohydrates=row['Carbohydrates'],
        protein=row['Protein'],
        fat=row['Fat'],
        sodium=row['Sodium'],
        sugar=row['Sugar'],
        timestamp=row['Date']
    )
    food.save()

    user = User.nodes.first()  
    user.has_consumed.connect(food)  

print("Success populating food data...")

Success populating food data...


In [19]:
for index, row in physical_activity.iterrows():
    unique_id = str(uuid.uuid4())

    activity = PhysicalActivity(
        _id=unique_id,
        daily_steps=row['Daily Steps'],
        distance=row['Distance (km)'],
        calories_burned=row['Calories Burned'],
        timestamp=row['Timestamp']
    )
    activity.save()

    user = User.nodes.first()  
    user.has_done.connect(activity)  

print("Success populating physical activity data...")

Success populating physical activity data...


In [20]:
for index, row in physiological_parameter.iterrows():
    unique_id = str(uuid.uuid4())

    physiology = PhysiologicalParameter(
        _id=unique_id,
        heart_rate=row['Heart Rate'],
        blood_pressure=row['Blood Pressure'],
        blood_oxygen=row['Blood Oxygen'],
        timestamp=row['Timestamp']
    )
    physiology.save()

    user = User.nodes.first()  
    user.has_monitored.connect(physiology) 

print("Success populating physiological parameter data...")

Success populating physiological parameter data...


In [21]:
for index, row in sleep_duration.iterrows():
    unique_id = str(uuid.uuid4())

    sleep = SleepDuration(
        _id=unique_id,
        hours=row['Hours'],
        minutes=row['Minutes'],
        timestamp=row['Timestamp']
    )
    sleep.save()

    user = User.nodes.first()  
    user.has_sleep.connect(sleep)  

print("Success populating sleep duration data...")

Success populating sleep duration data...
